<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Библиотеки" data-toc-modified-id="Библиотеки-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Библиотеки</a></span></li><li><span><a href="#Получение-данных" data-toc-modified-id="Получение-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Получение данных</a></span></li><li><span><a href="#Лишние-данные" data-toc-modified-id="Лишние-данные-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Лишние данные</a></span></li><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a href="#Разделение-на-выборки" data-toc-modified-id="Разделение-на-выборки-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Разделение на выборки</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#CatBoost" data-toc-modified-id="CatBoost-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>CatBoost</a></span></li><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Тестирование</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект по классификации комментариев  

Обучить модель классифицировать комментарии на позитивные и негативные. В  распоряжении набор данных с разметкой о токсичности правок.

Построить модель со значением метрики качества *F1* не меньше 0.75. 

## Подготовка

### Библиотеки

In [1]:
!pip install nltk
!pip install catBoost
!pip install lightgbm
!python -m spacy download en_core_web_sm

     -------------------------------------- 13.6/13.6 MB 903.6 kB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
import re
import nltk
import spacy
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier
from nltk.corpus import stopwords
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sheyk\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sheyk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Получение данных

In [4]:
df = pd.read_csv('toxic_comments.csv')

In [5]:
df

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
...,...,...,...
159287,159446,""":::::And for the second time of asking, when ...",0
159288,159447,You should be ashamed of yourself \n\nThat is ...,0
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,159449,And it looks like it was actually you who put ...,0


Узнаем размер данных

In [6]:
df.shape

(159292, 3)

### Лишние данные

В данных есть неинформативный столбец `Unnamed: 0`, удалим его

In [7]:
df = df.drop('Unnamed: 0', axis=1)

Если в столбце `text` есть объекты с русскими символами, то удалим их.

In [8]:
def checking_russian(x):
    if bool(re.search('[а-яА-ЯёЁ]', x)):
        return 'russian'
    else:
        return (' '.join(re.sub(r'[^a-zA-Z]', ' ', x).split())).lower()

In [9]:
df['text'] = df['text'].apply(checking_russian)

In [10]:
df[df['text'] == 'russian'].shape

(259, 2)

In [11]:
df = df[df['text'] != 'russian']

In [12]:
df.loc[0,'text']

'explanation why the edits made under my username hardcore metallica fan were reverted they weren t vandalisms just closure on some gas after i voted at new york dolls fac and please don t remove the template from the talk page since i m retired now'

**Вывод**

- Удалено 259 объектов, в которых фигурировались русские символы.
- Удалили все знаки препинания, удали оставили только английские символы.

### Лемматизация

Для обучения моделей нужно лемматизировать наши данные - привести слова в тексте в начальный вид и к нижнему регистру.

In [13]:
disabled_pipes = [ "parser",  "ner"]
nlp = spacy.load('en_core_web_sm', disable=disabled_pipes)

In [14]:
%%time
df["lemmatized_text"] = df["text"].apply(lambda x: " ".join([token.lemma_ for token in nlp(x)]))

Wall time: 12min 16s


In [15]:
df.loc[0,'lemmatized_text']

'explanation why the edit make under my username hardcore metallica fan be revert they weren t vandalism just closure on some gas after I vote at new york doll fac and please don t remove the template from the talk page since I m retire now'

In [16]:
df.loc[0,'text']

'explanation why the edits made under my username hardcore metallica fan were reverted they weren t vandalisms just closure on some gas after i voted at new york dolls fac and please don t remove the template from the talk page since i m retired now'

**Вывод**

Лемматизация прошла успешно.

### Разделение на выборки

Разделим наши данные на 3 выборки: 

- Обучающая выборка: 75%
- Валидационная выборка: 12.5%
- Тестовая выборка: 12.5%

In [17]:
RANDOM_STATE = 12345

In [18]:
train_df, temp_df = train_test_split(df, random_state=RANDOM_STATE, test_size=0.25)
valid_df, test_df = train_test_split(temp_df, random_state=RANDOM_STATE, test_size=0.5)

In [19]:
features_train = train_df['lemmatized_text'] 
target_train = train_df['toxic']

features_valid = valid_df['lemmatized_text'] 
target_valid = valid_df['toxic']

features_test = test_df[['lemmatized_text'] ]
target_test = test_df['toxic']

**Вывод**

Успешно разделили наши данные на выборки.

## Обучение

### CatBoost

Первой моделью будет CatBoost, а именно версия CatBoostClassifier, в данной модели есть такой параметр при обучении как text_features, который позволяет обрабатывать текстовую информацию без внешних преобразований.

In [20]:
%%time
model_catboost = CatBoostClassifier(verbose=100, learning_rate=0.7, early_stopping_rounds=200, eval_metric="F1")
model_catboost.fit(train_df[['lemmatized_text']], train_df[['toxic']], eval_set=(valid_df[['lemmatized_text']], valid_df[['toxic']]), text_features=['lemmatized_text'])

0:	learn: 0.6851816	test: 0.6964892	best: 0.6964892 (0)	total: 262ms	remaining: 4m 21s
100:	learn: 0.8273729	test: 0.7727992	best: 0.7749273 (69)	total: 7.39s	remaining: 1m 5s
200:	learn: 0.8578275	test: 0.7730384	best: 0.7749273 (69)	total: 14.3s	remaining: 57s
300:	learn: 0.8788812	test: 0.7764335	best: 0.7775151 (276)	total: 21.4s	remaining: 49.7s
400:	learn: 0.8962474	test: 0.7789142	best: 0.7795544 (384)	total: 28.5s	remaining: 42.6s
500:	learn: 0.9101333	test: 0.7793501	best: 0.7800786 (495)	total: 35.5s	remaining: 35.4s
600:	learn: 0.9225194	test: 0.7836349	best: 0.7841387 (588)	total: 42.4s	remaining: 28.2s
700:	learn: 0.9327958	test: 0.7816754	best: 0.7841802 (607)	total: 49.6s	remaining: 21.2s
800:	learn: 0.9410560	test: 0.7853595	best: 0.7854716 (795)	total: 56.6s	remaining: 14.1s
900:	learn: 0.9480801	test: 0.7866284	best: 0.7875621 (899)	total: 1m 3s	remaining: 7s
999:	learn: 0.9549496	test: 0.7860125	best: 0.7875621 (899)	total: 1m 10s	remaining: 0us

bestTest = 0.7875620

**Вывод**

Модель CatBoostClassifier была обучена, получили следующие результаты:

-  Время обучения: 1min 24s
-  Метрика F1: 0.7737306843267109

### LogisticRegression

Второй будем использовать модель LogisticRegression, для её обучения будем использовать метод TF-IDF для работы с текстом.

In [21]:
stopwords = set(stopwords.words('english')) 

In [22]:
%%time
model_regression = Pipeline(
    [('tf', TfidfVectorizer(stop_words=stopwords, ngram_range=(1, 1))), 
     ('reg', LogisticRegression(random_state=RANDOM_STATE, class_weight='balanced', solver='liblinear', max_iter=1000, verbose=1, C=1.0))
    ])

Wall time: 0 ns


In [23]:
%%time
model_regression.fit(features_train, target_train)

[LibLinear]Wall time: 6.01 s


Pipeline(steps=[('tf',
                 TfidfVectorizer(stop_words={'a', 'about', 'above', 'after',
                                             'again', 'against', 'ain', 'all',
                                             'am', 'an', 'and', 'any', 'are',
                                             'aren', "aren't", 'as', 'at', 'be',
                                             'because', 'been', 'before',
                                             'being', 'below', 'between',
                                             'both', 'but', 'by', 'can',
                                             'couldn', "couldn't", ...})),
                ('reg',
                 LogisticRegression(class_weight='balanced', max_iter=1000,
                                    random_state=12345, solver='liblinear',
                                    verbose=1))])

In [24]:
pred_regression = model_regression.predict(features_valid)

In [25]:
display(f1_score(pred_regression, target_valid))

0.7549584133077414

**Вывод**

Модель LogisticRegression была обучена, получили следующие результаты:

-  Время обучения: 8.19 s
-  Метрика F1: 0.7549584133077414

### Тестирование

In [26]:
predict_catboost = model_catboost.predict(features_test)

In [27]:
display(f1_score(predict_catboost, target_test))

0.7737306843267109

Результат метрики F1 выше 0.75.

## Выводы

- Были удалены объекты с русскими символами.
- Данные были лемматизированы.
- Лучшая модель CatBoostClassifier с результатами:  
 -  Время обучения: 1min 24s
 -   Метрика F1 на тестовой выборке: 0.7737306843267109

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны